In [67]:
import numpy as np
import pandas as pd
#ast convert string into list
import ast
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split


In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [5]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [6]:
movies.shape

(4803, 20)

In [7]:
credits.shape

(4803, 4)

# Merging movies and credits

In [8]:
movies = movies.merge(credits, on="title")

In [9]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [10]:
movies.shape

(4809, 23)

In [11]:
#movie_id
#title
#overview
#genres
#keywords
#cast
#crew
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [12]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [13]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [16]:
movies.duplicated().sum()

0

In [17]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action' , 'Adventure' , 'Fantasy' , 'Science Fiction']

In [19]:
movies.iloc[0].keywords
#only need to pick names

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [20]:
movies.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [21]:
movies.iloc[0].crew

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [22]:
def convert(data):
    L=[]
    for i in ast.literal_eval(data):
        L.append(i['name'])
    return L


In [23]:
movies['genres'] = movies['genres'].apply(convert)

In [24]:
movies['keywords'] = movies['keywords'].apply(convert)

In [25]:
movies['cast'] = movies['cast'].apply(convert)

In [26]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [27]:
movies['cast'][0]

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

In [28]:
def fatchdirector(data):
    L=[]
    for i in ast.literal_eval(data):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [29]:
movies['crew'] = movies['crew'].apply(fatchdirector)

In [30]:
movies['crew'][0:2]

0     [James Cameron]
1    [Gore Verbinski]
Name: crew, dtype: object

In [31]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]


In [32]:
def removespace(text):
    l = []
    for i in text:
        l.append(i.replace(" ",""))
    return l

In [33]:
movies['genres'] = movies['genres'].apply(removespace)
movies['keywords'] = movies['keywords'].apply(removespace)
movies['cast'] = movies['cast'].apply(removespace)
movies['crew'] = movies['crew'].apply(removespace)

In [34]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [35]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [36]:
movies['overview'][0:2]

0    [In, the, 22nd, century,, a, paraplegic, Marin...
1    [Captain, Barbossa,, long, believed, to, be, d...
Name: overview, dtype: object

In [37]:
movies['tag'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [38]:
Newmovies = movies.drop(columns=['overview' , 'genres' , 'keywords' , 'cast' , 'crew'])

In [39]:
Newmovies.head(2)

,movie_id,title,tag
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."


In [40]:
Newmovies['tag'] = Newmovies['tag'].apply(lambda x: " ".join(x))

In [41]:
Newmovies['tag'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [42]:
Newmovies['tag'] = Newmovies['tag'].apply(lambda x: x.lower())

In [43]:
Newmovies['tag'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [44]:
#import stemmer from nltk

In [45]:
from nltk.stem.porter import PorterStemmer
PS = PorterStemmer()

In [46]:
def stemmerporter(text):
    l=[]
    for i in text.split():
        l.append(PS.stem(i))
    return " ".join(l)

In [47]:
Newmovies['tag'] = Newmovies['tag'].apply(stemmerporter)

In [48]:
Newmovies['tag'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

# Text Vectorization


In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(Newmovies['tag'])


In [90]:
tfidf_matrix

<4806x33160 sparse matrix of type '<class 'numpy.float64'>'
	with 194243 stored elements in Compressed Sparse Row format>

In [97]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)


In [98]:
def recommend_movies(movie_title, cosine_sim=cosine_sim, N=10):
    # Get the index of the movie
    idx = Newmovies[Newmovies['title'] == movie_title].index[0]

    # Get the cosine similarity scores of all movies with the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N similar movies (excluding the input movie itself)
    sim_scores = sim_scores[1:N+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top N recommended movie titles
    return Newmovies['title'].iloc[movie_indices]


In [108]:
recommended_movies = recommend_movies('Avatar')
print(recommended_movies)


2409                         Aliens
3731                  Falcon Rising
582             Battle: Los Angeles
778                       Meet Dave
47          Star Trek Into Darkness
3609                      Apollo 18
539                      Titan A.E.
1204                      Predators
1216    Aliens vs Predator: Requiem
942                The Book of Life
Name: title, dtype: object


# Word Embeddings (Word2Vec)

In [104]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

# Tokenize and preprocess your movie descriptions (assuming 'tag' column contains preprocessed text)
tokenized_descriptions = [nltk.word_tokenize(tag) for tag in Newmovies['tag']]

# Train a Word2Vec model
model = Word2Vec(tokenized_descriptions, vector_size=1000, window=5, min_count=1, sg=0)

# Function to get movie recommendations based on Word2Vec similarity
def recommend_movies_word2vec(movie_title, model=model, topn=10):
    # Convert input title to lowercase
    movie_title = movie_title.lower()

    # Check if the title exists in the vocabulary
    if movie_title not in model.wv:
        return []

    similar_movies = model.wv.most_similar(movie_title, topn=topn)
    return [movie for movie, score in similar_movies]

# Example usage
input_movie_title = 'Batman'  # You can change this to any movie title in your dataset
recommended_movies_word2vec = recommend_movies_word2vec(input_movie_title)
print(recommended_movies_word2vec)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['weapon', 'militari', 'church', 'broadway', 'grave', 'light', 'parti', 'theft', 'wound', 'priest']


# Jaccard Similarity

In [107]:
# Function to calculate Jaccard Similarity between two sets
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Function to recommend movies based on Jaccard Similarity of genres and keywords
def recommend_movies_jaccard(movie_title, threshold=0.2, topn=10):
    target_movie = Newmovies[Newmovies['title'] == movie_title]

    # Calculate Jaccard Similarity with all other movies
    similarities = []
    for _, row in Newmovies.iterrows():
        similarity = jaccard_similarity(set(target_movie['title'].values[0]), set(row['title'])) + \
                     jaccard_similarity(set(target_movie['title'].values[0]), set(row['title']))
        similarities.append((row['title'], similarity))

    # Sort movies by similarity and filter by threshold
    similar_movies = sorted(similarities, key=lambda x: x[1], reverse=True)
    similar_movies = [(title, score) for title, score in similar_movies if score >= threshold]

    # Return the top N similar movies
    return [title for title, _ in similar_movies[:topn]]

# Example usage
recommended_movies_jaccard = recommend_movies_jaccard('Tangled')
print(recommended_movies_jaccard)


['Tangled', 'Triangle', 'Strangerland', 'Tadpole', 'The Island', 'Highlander', 'Thunderball', 'The Wailing', 'Sling Blade', 'The Calling']
